# model 2

In [ ]:
# Model pertama
model_name_1 = "openai/clip-vit-base-patch32"
model_1 = CLIPModel.from_pretrained(model_name_1)
processor_1 = CLIPProcessor.from_pretrained(model_name_1)

# Model kedua
model_name_2 = "openai/clip-vit-large-patch14"
model_2 = CLIPModel.from_pretrained(model_name_2)
processor_2 = CLIPProcessor.from_pretrained(model_name_2)

device = "cuda" if torch.cuda.is_available() else "cpu"
model_1.to(device)
model_2.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 768)
      (position_embedding): Embedding(77, 768)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e

In [ ]:
folder_path = "/content/drive/MyDrive/DATA PENGPOL/data/train normalisasi/Accident"
csv_path = "/content/clip_probabilities.csv"

data = pd.read_csv(csv_path)
assert 'Image Path' in data.columns and 'Best Description' in data.columns, "CSV harus memiliki kolom 'image_name' dan 'description'"
data['Image Path'] = data['Image Path'].apply(lambda x: os.path.join(folder_path, x))


In [ ]:
def process_inputs(processor, image_path, description):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(
        text=[description], images=image, return_tensors="pt", padding=True
    )
    return {k: v.to(device) for k, v in inputs.items()}


In [ ]:
results = []

for idx, row in data.iterrows():
    try:
        # Proses data untuk model 1
        inputs_1 = process_inputs(processor_1, row['Image Path'], row['Best Description'])
        outputs_1 = model_1(**inputs_1)
        score_1 = outputs_1.logits_per_image.item()

        # Proses data untuk model 2
        inputs_2 = process_inputs(processor_2, row['Image Path'], row['Best Description'])
        outputs_2 = model_2(**inputs_2)
        score_2 = outputs_2.logits_per_image.item()

        # Save results
        results.append({
            "Image Path": row['Image Path'],
            "Best Description": row['Best Description'],
            "score_model_1": score_1,
            "score_model_2": score_2
        })

    except Exception as e:
        print(f"Error processing {row['Image Path']}: {e}")


In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv("comparison_results.csv", index=False)
print(results_df.head())

                                          Image Path  \
0  /content/drive/MyDrive/DATA PENGPOL/data/train...   
1  /content/drive/MyDrive/DATA PENGPOL/data/train...   
2  /content/drive/MyDrive/DATA PENGPOL/data/train...   
3  /content/drive/MyDrive/DATA PENGPOL/data/train...   
4  /content/drive/MyDrive/DATA PENGPOL/data/train...   

                                    Best Description  score_model_1  \
0  A motorcycle accident involving multiple vehicles      31.020559   
1  A motorcycle accident involving multiple vehicles      30.557634   
2                           A car accident on a road      31.016859   
3         The car crashed into a pedestrian crossing      29.133537   
4                           A car accident on a road      28.771816   

   score_model_2  
0      21.107773  
1      21.148611  
2      21.187666  
3      24.147114  
4      24.681755  


In [ ]:
results_df['score_difference'] = results_df['score_model_2'] - results_df['score_model_1']
print(results_df[['Image Path', 'score_difference']].head())

                                          Image Path  score_difference
0  /content/drive/MyDrive/DATA PENGPOL/data/train...         -9.912786
1  /content/drive/MyDrive/DATA PENGPOL/data/train...         -9.409023
2  /content/drive/MyDrive/DATA PENGPOL/data/train...         -9.829193
3  /content/drive/MyDrive/DATA PENGPOL/data/train...         -4.986423
4  /content/drive/MyDrive/DATA PENGPOL/data/train...         -4.090061


In [ ]:
avg_score_model_1 = results_df['score_model_1'].mean()
avg_score_model_2 = results_df['score_model_2'].mean()
print(f"Average Score Model 1: {avg_score_model_1}")
print(f"Average Score Model 2: {avg_score_model_2}")

Average Score Model 1: 29.581415007455007
Average Score Model 2: 23.793130997249058


In [ ]:
import time

start_time = time.time()
for idx, row in data.iterrows():
    inputs_1 = process_inputs(processor_1, row['Image Path'], row['Best Description'])
    outputs_1 = model_1(**inputs_1)
print(f"Model 1 Time: {time.time() - start_time}")

start_time = time.time()
for idx, row in data.iterrows():
    inputs_2 = process_inputs(processor_2, row['Image Path'], row['Best Description'])
    outputs_2 = model_2(**inputs_2)
print(f"Model 2 Time: {time.time() - start_time}")

Model 1 Time: 102.64962816238403
Model 2 Time: 1194.5440151691437
